In [ ]:
#STEPS

# 1) Connect to Database
# 2) Load the JSON
# 2) Run the DS query to get the list of users who were onboarded during the given period


In [16]:
import pandas as pd
import psycopg2 as ps
import json
from datetime import datetime
import copy
import math
import requests
import glob
import os
from concurrent.futures import ThreadPoolExecutor

In [17]:
#CONNECT TO DB

conn = ps.connect(
    host='ldc-ios-prod.cluster-ro-c60xechvrdpu.ap-south-1.rds.amazonaws.com',
    port=5432,
    dbname="investorsdb",
    user="read_usrinvestors",
    password="FS4o#cjpe9H$yC73")

In [18]:
#ALL JSONS

MAIN_JSON = {
    "identity": None, 
    "type": "event", 
    "ts": None, 
    "evtName": None, 
    "evtData": None
}

INV_SIGNUP_S2S = {
    'Acquisition channel': 'None',
    'Identity': 'None',
    'Phone': 'None', 
    'Sourcing Channel': 'None',
    'Platform': 'Retail',
    'Status': '1',
    'Reason': 'None'
}

INV_LIVE_KYC_S2S = {
    'Identity': 'None',
    'Status': '1',
    'Reason': 'None'
}

INV_LEGAL_AUTH_S2S = {
    'Identity': 'None',
    'Status': '1',
    'Reason': 'None'
}

INV_LISTED_S2S = {
    'Identity': 'None',
    'Account Status': 'None'
}


INV_ADD_BANKACCOUNT_S2S = {
    'Identity': 'None',
    'Bank Name': 'None',
    'Status': '1',
    'Reason': 'None'
}


INV_FMPP_S2S = {
    'Identity': 'None',
    'Scheme ID': 'None',
    'FMPP Amount': 'None',
    'Maturity Amount': 'None',
    'Period': 'None',
    'ROI': 'None',
    'Maturity Date': 'None',
    'FMPP Type': 'None',
    'Auto Renew': 'None',
    'Payment Method': 'None',
    'Status': '1',
    'Reason': 'None',
    'Account Balance': 'None',
    'Maturity Timestamp': 'None'
}


INV_FMPP_S2S_BULK = {
    'Identity': 'None',
    'Scheme ID': 'None',
    'FMPP Amount': 'None',
    'Maturity Amount': 'None',
    'Period': 'None',
    'ROI': 'None',
    'Maturity Date': 'None',
    'FMPP Type': 'None',
    'Auto Renew': 'None',
    'Payment Method': 'None',
    'Status': '1',
    'Reason': 'None',
    'Account Balance': 'None',
    'Maturity Timestamp': 'None',
    'Order ID': 'None'
}

In [19]:
#DEFINE THE MONTH

months = {
    'OCT_2024': ['2024-10-24 18:30:00', '2024-10-27 18:29:59']
}
dates = months.values()

In [20]:
dates = list(months.values())[0]
dates
dates[0]

'2024-10-24 18:30:00'

In [21]:
#GET THE USER FROM THIS QUERY

def signup_ds(month, key, month_key):
    query = f"""select lc.user_id, date(la.created_date)  from lendenapp_customuser lc
                inner join lendenapp_user_source_group lusg on lc.id = lusg.user_id 
                inner join lendenapp_account la on la.user_source_group_id = lusg.id 
                where lusg.source_id = 7 and la.created_date between '{dates[0]}' and '{dates[1]}';
            """
    print(query)
    
    signup_query = pd.read_sql_query(query, con=conn)
    signup_query.to_csv(f'All New Data for CT/Users from CSV/SU/{month_key}_{key}.csv')

In [22]:
#RUN DS QUERIES

for key, value in months.items():
    print(f"STORING SIGNUP for {key}")
    signup_ds(value, 'SU', key)

STORING SIGNUP for OCT_2024
select lc.user_id, date(la.created_date)  from lendenapp_customuser lc
                inner join lendenapp_user_source_group lusg on lc.id = lusg.user_id 
                inner join lendenapp_account la on la.user_source_group_id = lusg.id 
                where lusg.source_id = 7 and la.created_date between '2024-10-24 18:30:00' and '2024-10-27 18:29:59';
            


/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/2809698353.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  signup_query = pd.read_sql_query(query, con=conn)


In [23]:
#GET ALL USERS FOR THAT MONTH

def list_files_func(folder_path):
    return glob.glob(os.path.join(folder_path, '*'))

def create_data_csvs(folder):
    list_files = list_files_func(f'All New Data for CT/Users from CSV/{folder}')

    all_users = []

    for index in range(len(list_files)):
        print(list_files[index])
        data = pd.read_csv(list_files[index])
        all_users.extend(list(data['user_id']))
    
    return all_users

In [24]:
users = create_data_csvs('SU')
users = tuple(users)

All New Data for CT/Users from CSV/SU/OCT_2024_SU.csv


In [25]:
#ALL DATA

def get_data_for_signup(data):
    query = f"""
                SELECT 
                lc.user_id as "Identity", 
                (la.created_date) as ts,
                lc.mobile_number as "Phone", 
                ls.source_name as "Sourcing Channel" 
                FROM lendenapp_customuser lc
                INNER JOIN lendenapp_user_source_group lusg ON lc.id = lusg.user_id
                INNER JOIN lendenapp_account la ON la.user_source_group_id = lusg.id
                INNER JOIN lendenapp_source ls ON ls.id = lusg.source_id
                WHERE lc.user_id IN {data} AND lusg.source_id = 7
            """
    
    signups = pd.read_sql_query(query, con=conn)
    return signups


def get_data_for_kyc(data, dates):
    query = f"""
                SELECT lt.checklist, lt.history_date, lc.user_id, lt.created_by_id 
                FROM lendenapp_historicaltask lt
                LEFT JOIN lendenapp_user_source_group lusg ON lt.user_source_group_id = lusg.id
                LEFT JOIN lendenapp_customuser lc ON lc.id = lusg.user_id
                WHERE lusg.source_id = 7 AND lt.history_date between '{dates[0]}' and '{dates[1]}';
            """
    kycs = pd.read_sql_query(query, con=conn)
    kycs.to_csv('KYC History Data.csv')

    kyc = {'Identity': [], 'ts': []}
    data_for_kyc = pd.read_csv('KYC History Data.csv')

    checklist = list(data_for_kyc['checklist'])
    user_id = list(data_for_kyc['user_id'])
    ts = list(data_for_kyc['history_date'])

    final = []
    for index in range(len(checklist)):
        if type(checklist[index]) != float:  # Skip NaN values
            current_checklist = checklist[index]
            try:
                # Convert string to valid JSON object
                current_checklist = current_checklist.replace("'", '"')  # Convert single quotes to double quotes if needed
                current_checklist = json.loads(current_checklist)  # Convert to a dictionary

                # Check if 'completed_steps' exists and if the last step is 'LIVE KYC'
                if current_checklist and 'completed_steps' in current_checklist and len(current_checklist['completed_steps'])>0 and current_checklist['completed_steps'][-1] == 'LIVE KYC':
                    kyc['Identity'].append(user_id[index])
                    kyc['ts'].append(ts[index])
                    final.append(user_id[index])

            except Exception as e:
                print(f"Error decoding JSON for user_id {user_id[index]}: {e}")
                continue  # Skip rows with invalid JSON

    kyc_df = pd.DataFrame(kyc, columns=['Identity', 'ts'])
    
    return kyc_df


def get_data_for_legal(data, dates):
    query = f"""
                SELECT lt.checklist, lt.history_date, lc.user_id, lt.created_by_id from lendenapp_historicaltask lt
                LEFT JOIN lendenapp_user_source_group lusg ON lt.user_source_group_id = lusg.id
                LEFT JOIN lendenapp_customuser lc ON lc.id = lusg.user_id
                where lusg.source_id = 7 and lt.history_date between '{dates[0]}' and '{dates[1]}';
            """
    legs = pd.read_sql_query(query, con=conn)
    legs.to_csv('Legal History Data.csv')

    legal = {'Identity': [], 'ts':[]}
    data_for_legal = pd.read_csv('Legal History Data.csv')

    checklist = list(data_for_legal['checklist'])
    user_id = list(data_for_legal['user_id'])
    ts = list(data_for_legal['history_date'])


    final = []
    for index in range(len(checklist)):
        if type(checklist[index]) != float:  # Skip NaN values
            current_checklist = checklist[index]
            try:
                # Convert string to valid JSON object
                current_checklist = current_checklist.replace("'", '"')  # Convert single quotes to double quotes if needed
                current_checklist = json.loads(current_checklist)  # Convert to a dictionary

                # Check if 'completed_steps' exists and if the last step is 'LIVE KYC'
                if current_checklist and 'completed_steps' in current_checklist and len(current_checklist['completed_steps'])>0 and current_checklist['completed_steps'][-1] == 'LEGAL AUTHORIZATION':
                    legal['Identity'].append(user_id[index])
                    legal['ts'].append(ts[index])
                    final.append(user_id[index])

            except Exception as e:
                print(f"Error decoding JSON for user_id {user_id[index]}: {e}")
                continue  # Skip rows with invalid JSON

    legal = pd.DataFrame(legal, columns=['Identity', 'ts'])
    return legal

def get_data_for_bank(data, dates):
    query = f"""
                SELECT lba.created_date as "ts", 
                lb.name as "Bank Name",
                lc.user_id as "Identity"
                FROM lendenapp_bankaccount lba
                LEFT JOIN lendenapp_bank lb ON lb.id = lba.bank_id
                LEFT JOIN lendenapp_customuser lc ON lc.id = lba.user_id
                LEFT JOIN lendenapp_user_source_group lusg ON lba.user_source_group_id = lusg.id
                where lusg.source_id = 7 and lba.created_date between '{dates[0]}' and '{dates[1]}'
            """
    banks = pd.read_sql_query(query, con=conn)
    return banks


def get_data_for_listed(data):
    query = f"""
                SELECT lba.created_date AS "ts", 
                lc.user_id AS "Identity"
                FROM lendenapp_bankaccount lba
                INNER JOIN lendenapp_customuser lc ON lc.id = lba.user_id
                INNER JOIN lendenapp_user_source_group lusg ON lba.user_source_group_id = lusg.id
                INNER JOIN lendenapp_account la ON lusg.id = la.user_source_group_id
                WHERE lusg.source_id = 7 AND la.listed_date between '2024-10-25' and '2024-10-27';
            """
    listeds = pd.read_sql_query(query, con=conn)

    listeds['Account Status'] = 'LISTED'
    return listeds

#

def get_manual_lending_data(data, dates):
    print(dates)
    query = f"""
                SELECT lc.user_id as "Identity",
                tis.urn_id as "Scheme ID", tis.investment_amount as "FMPP Amount",
                tis.expected_temp_interest_repayment_sum as "Maturity Amount", tsm.tenure as "Period",
                tis.expected_roi as "ROI", tis.maturity_date as "Maturity Date",
                tis.auto_renew as "Auto Renew", 
                EXTRACT(EPOCH FROM tis.maturity_date) as "Maturity Timestamp", 
                tis.order_id as "Order ID",
                tis.created_dtm as "ts"
                FROM fmpp.t_investor_scheme tis 
                INNER JOIN fmpp.t_investors ts ON tis.investor_id = ts.id 
                INNER JOIN lendenapp_customuser lc ON lc.user_id = ts.source_id
                INNER JOIN fmpp.t_scheme_master tsm ON tsm.id = tis.scheme_master_id
                INNER JOIN lendenapp_user_source_group lusg ON lusg.user_id = lc.id
                WHERE tis.order_id IS NOT null
                AND tis.partner_code_id = 49 AND 
                tis.created_dtm between '2024-10-24 18:30:00' and '2024-10-27 18:29:59'
                AND tis.deleted is null AND lusg.source_id = 7;
            """
    
    ml_schemes = pd.read_sql_query(query, con=conn)
    
    ml_schemes_grouped = ml_schemes.groupby('Order ID').agg({
            'Identity': 'first',
            'Auto Renew': 'first',
            'ts': 'first',
            'Order ID': 'first',
            'FMPP Amount': 'sum',
            'ROI': 'mean',
            'Maturity Date': 'max',
            'Scheme ID': 'first',
            'Period': 'first',
            'Maturity Amount': 'first',
            'Maturity Timestamp': 'max'
        }).reset_index(drop=True)

    ml_schemes_grouped['FMPP Type'] = 'BULK_LENDING'

    ml_schemes_grouped.to_csv('xyz.com')
        
    return ml_schemes_grouped

def get_scheme_data(data, dates):
    print(dates)
    query = f"""
                SELECT lc.user_id as "Identity",
                tis.urn_id as "Scheme ID", tis.investment_amount as "FMPP Amount",
                tis.expected_temp_interest_repayment_sum as "Maturity Amount", tsm.tenure as "Period",
                tis.expected_roi as "ROI", tis.maturity_date as "Maturity Date",
                tis.auto_renew as "Auto Renew", 
                EXTRACT(EPOCH FROM tis.maturity_date) as "Maturity Timestamp", 
                tis.created_dtm as "ts",
                tmp.key_2 as "FMPP Type"
                FROM fmpp.t_investor_scheme tis
                INNER JOIN fmpp.t_investors ts ON tis.investor_id = ts.id 
                INNER JOIN lendenapp_customuser lc ON lc.user_id = ts.source_id
                INNER JOIN fmpp.t_mst_parameter tmp ON tmp.id = tis.investment_type_id
                INNER JOIN fmpp.t_scheme_master tsm ON tsm.id = tis.scheme_master_id
                INNER JOIN lendenapp_user_source_group lusg ON lusg.user_id = lc.id
                WHERE tis.order_id IS null
                AND tis.partner_code_id = 49 
                AND tis.created_dtm between '2024-10-24 18:30:00' and '2024-10-27 18:29:59'
                AND tis.deleted is null AND lusg.source_id = 7;
            """
    
    schemes = pd.read_sql_query(query, con=conn)

    schemes.to_csv('abc.com')

    return schemes


#'2022-07-31 18:30:00' and '2022-12-31 18:29:59'
#'2022-12-31 18:30:00' and '2023-06-30 18:29:59'
#'2023-06-30 18:30:00' and '2023-12-31 18:29:59'
#'2023-12-31 18:30:00' and '2024-06-30 18:29:59'
#'2024-06-30 18:30:00' and '2024-10-14 18:29:59'

In [28]:
def create_data_csvs(folder, function, data, dates, file_name, dates_needed):
    if dates_needed:
        given_data=function(data, dates)
    else:
        given_data=function(data)
    file_name_join = file_name+'DATA'
    given_data.to_csv(f'All New Data for CT/Users from CSV/{folder}/{file_name_join}.csv')

In [29]:
create_data_csvs('SU', get_data_for_signup, users, dates, 'SU', dates_needed=False)
create_data_csvs('KYC', get_data_for_kyc, users,dates, 'KYC', dates_needed=True)
create_data_csvs('LA', get_data_for_legal, users,dates, 'LA', dates_needed=True)
create_data_csvs('BA', get_data_for_bank, users,dates, 'BA', dates_needed=True)
create_data_csvs('LI', get_data_for_listed, users,dates, 'LI', dates_needed=False)
create_data_csvs('FMPP/BL', get_manual_lending_data, users, dates, 'BL', dates_needed=True)
create_data_csvs('FMPP', get_scheme_data, users, dates, 'FMPP', dates_needed=True)

/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/159060056.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  signups = pd.read_sql_query(query, con=conn)
/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/159060056.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  kycs = pd.read_sql_query(query, con=conn)
/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/159060056.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  legs = pd.read_sql_query(query, con=conn)
/var/folders/ll/lgf

['2024-10-24 18:30:00', '2024-10-27 18:29:59']


/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/159060056.py:157: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ml_schemes = pd.read_sql_query(query, con=conn)


['2024-10-24 18:30:00', '2024-10-27 18:29:59']


/var/folders/ll/lgfhb72d6_q0t0_xmyt0d8cc0000gp/T/ipykernel_4775/159060056.py:202: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schemes = pd.read_sql_query(query, con=conn)


In [30]:
def convert_to_timestamp(given_time):
    epoch_timestamp = int(given_time.timestamp())
    return epoch_timestamp

In [31]:
def create_data(dataframe, event_json, main_json, event_name, columns_to_update):
    final_data = []
    all_users = []
    p_count=0
    n_count=0
    
    for index in range(len(dataframe)):
        try:
            if dataframe['Identity'][index]:
                current_json = copy.deepcopy(main_json)
                new_json = copy.deepcopy(event_json)
                new_json.update(
                    {
                        column: dataframe[column][index]
                        for column in columns_to_update
                    }
                )
                current_json.update(
                    {
                        'identity': dataframe['Identity'][index],
                        'ts': convert_to_timestamp(datetime.fromisoformat(dataframe['ts'][index])),
                        'evtName': event_name,
                        'evtData': new_json,
                    }
                )
                if 'Auto Renew' in new_json:
                    new_json.update({
                        'Auto Renew': str(new_json['Auto Renew'])
                    })
                if current_json['identity']:
                    final_data.append(current_json)
                    if current_json['identity'] not in all_users:
                        all_users.append(current_json['identity'])

        except Exception as e:
            print(e, index, dataframe['Identity'][index])

    return final_data, len(all_users)

In [32]:
def create_final_pack(event, event_json, main_json, event_name, columns):
    list_files = list_files_func(f'All New Data for CT/Users from CSV/{event}')
    data_json = {}
    user_json = {}
    variable_key_name = f'{event}_month'
    try:
        for index in range(len(list_files)):
            if list_files[index][-8:] == 'DATA.csv':
                current_csv = pd.read_csv(list_files[index])
                current_month, all_users = create_data(current_csv, event_json, main_json, event_name, columns)
                variable_key = f'{variable_key_name}_{str(index+1)}'
                data_json[variable_key] = current_month
                user_json[variable_key] = all_users
    except Exception as e:
        print(e, list_files[index])
    
    return data_json, user_json

In [33]:
signup_json = create_final_pack('SU', INV_SIGNUP_S2S, MAIN_JSON, 'INV SignUp S2S', ['Identity', 'Phone', 'Sourcing Channel'])
kyc_json = create_final_pack('KYC', INV_LIVE_KYC_S2S, MAIN_JSON, 'INV Live KYC S2S', ['Identity'])
la_json = create_final_pack('LA', INV_LEGAL_AUTH_S2S, MAIN_JSON, 'INV Legal Authorization S2S', ['Identity'])
bank_json = create_final_pack('BA', INV_ADD_BANKACCOUNT_S2S, MAIN_JSON, 'INV Bank Account S2S', ['Identity', 'Bank Name'])
listed_json = create_final_pack('LI', INV_LISTED_S2S, MAIN_JSON, 'INV Listed S2S', ['Identity', 'Account Status'])
bl_json = create_final_pack('FMPP/BL', INV_FMPP_S2S_BULK, MAIN_JSON, 'INV CREATE FMPP SCHEME S2S', ['Identity', 'Scheme ID', 'FMPP Amount','Maturity Amount','Period','ROI', 'Maturity Date', 'FMPP Type', 'Auto Renew', 'Maturity Timestamp', 'Order ID'])
al_json = create_final_pack('FMPP', INV_FMPP_S2S, MAIN_JSON, 'INV CREATE FMPP SCHEME S2S', ['Identity', 'Scheme ID', 'FMPP Amount','Maturity Amount','Period','ROI', 'Maturity Date', 'FMPP Type', 'Auto Renew', 'Maturity Timestamp'])

In [34]:
def batch_split(json_data, max_record):
    highest = len(json_data)

    batches = math.ceil(highest / max_record)
    if batches>0:
        size_per_batch = highest // batches

        return size_per_batch
    
def create_batches(ct_json, batch_size):
    final = []
    for index in range(0, len(ct_json), batch_size):
        x={}
        x["d"]=(ct_json[index:index+batch_size])
        final.append(x)
    
    return final

In [35]:
class CleverTap:
    def __init__(self, account_id, passcode):
        self.account_id = account_id
        self.passcode = passcode

    @property
    def headers(self):
        return {
            'X-CleverTap-Account-Id': self.account_id,
            'X-CleverTap-Passcode': self.passcode,
            'Content-Type': 'application/json; charset=utf-8',
        }
    def upload_events(self, url, data):
        data = str(data)
        headers = self.headers
        response = requests.post(url=url, headers=headers, data=data)
        return response

    def upload_user(self, url, data):
        data = str(data)
        headers = self.headers
        response = requests.post(url=url, headers=headers, data=data)
        return response

c=CleverTap('R95-W74-766Z', 'ERO-QMA-IEUL')

In [36]:
GLOB = 0
def send_batch(batch):
    global c
    global GLOB
    url = "CleverTap API endpoint"
    headers = {"Content-Type": "application/json", "Authorization": "Bearer YOUR_API_KEY"}

    response=c.upload_events("https://in1.api.clevertap.com/1/upload", batch)
    if response.status_code == 200:
        print("Batch successfully uploaded")
    else:
        print(f"Failed to upload batch: {response.text}")

In [37]:
def final_push(final):      
    with ThreadPoolExecutor(max_workers=15) as executor:
        executor.map(send_batch, final)


def push_data(data):
    # for data in data_array:
    batch_size = batch_split(data, 1000)
    if data and batch_size and batch_size>0:
        final = create_batches(ct_json=data, batch_size=batch_size)
        final_push(final)

In [58]:
final_jsons = [signup_json[0], kyc_json[0], la_json[0], bank_json[0], al_json[0], bl_json[0]]

for json in final_jsons:
    for key, value in json.items():
        push_data(value)

Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploadedBatch successfully uploaded

Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded
Batch successfully uploaded


In [38]:
def get_stats(al_json):
    users = set()
    amount = 0
    for i in range(len(al_json[0]['FMPP_month_2'])):
        current=al_json[0]['FMPP_month_2'][i]['evtData']
        users.add(current['Identity'])
        amount+=current['FMPP Amount']

    return len(users), amount, len(al_json[0]['FMPP_month_2'])


def get_stats_bl(bl_json):
    users = set()
    amount = 0
    for i in range(len(bl_json[0]['FMPP/BL_month_1'])):
        current=bl_json[0]['FMPP/BL_month_1'][i]['evtData']
        users.add(current['Identity'])
        amount+=current['FMPP Amount']

    return len(users), amount, len(bl_json[0]['FMPP/BL_month_1'])

In [62]:
print("SIGN UPS: ", "\t", len(signup_json[0]['SU_month_1']))
print("LIVE KYC: ", "\t", len(kyc_json[0]['KYC_month_1']))
print("LEGAL AUTH: ", "\t", len(la_json[0]['LA_month_1']))
print("BANK ACCOUNT: ", "\t", len(bank_json[0]['BA_month_1']))
print("USER LISTED: ", "\t", len(listed_json[0]['LI_month_1']))
print("\n")
print("CR SCHEMES", "\t", "SCHEMES AMOUNT USERS")
print("BULK LENDING: ","\t", get_stats_bl(bl_json))
print("OTHER SCHEMES:  ", get_stats(al_json))

SIGN UPS:  	 4800
LIVE KYC:  	 2593
LEGAL AUTH:  	 2491
BANK ACCOUNT:  	 2532
USER LISTED:  	 2537


CR SCHEMES 	 SCHEMES AMOUNT USERS
BULK LENDING:  	 (249, 1781718.0, 818)
OTHER SCHEMES:   (16, 1667492.0, 16)


In [63]:
import json

In [69]:
x=json.dumps({'x':Decimal(1.54)})

TypeError: Object of type Decimal is not JSON serializable

In [66]:
from decimal import Decimal, decimal_d

In [73]:
def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)
    raise TypeError("Object of type Decimal is not JSON serializable")

# Convert event_data to JSON, handling Decimal values
params = {
    'status': 'SUCCESS',
    'event_data': json.dumps(event_data, default=decimal_default)
}

In [72]:
event_data = {'x': Decimal('4.59'), 'y':'Glen'}

In [74]:
params

{'status': 'SUCCESS', 'event_data': '{"x": 4.59, "y": "Glen"}'}